In [ ]:
import sys, time, os, asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy.units as u
from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from tqdm import tqdm
from datetime import datetime

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
def add_timestamp(data):
    """
    Adds a correct timestamp column in UTC format to the provided data if
    not present.

    Parameters
    ----------
    data : DataFrame
        The data to which the timestamp will be added.

    Returns
    -------
    DataFrame
        The data with the added 'snd_timestamp_utc' column.
    """
    if "snd_timestamp_utc" not in data.columns:
        data["snd_timestamp_utc"] = Time(
            data["private_sndStamp"], format="unix_tai"
        ).unix
    return data


def get_data(events_frame, client_usdf, client_idf, train=False):
    table_dict = {
        "m1m3_hp_actuator": "lsst.sal.MTM1M3.hardpointActuatorData",
        # "m1m3_hp_monitor": "lsst.sal.MTM1M3.hardpointMonitorData",
        # "mtmount_elevation": "lsst.sal.MTMount.elevation",
        # "mtmount_azimuth": "lsst.sal.MTMount.azimuth",
        # "m1m3_ims": "lsst.sal.MTM1M3.imsData"
    }
    query_dict = {}
    for key in table_dict.keys():
        query_dict[key] = []
    for j, time in tqdm(enumerate(events_frame["time"])):
        event_time = Time(events_frame["time"][j], format="unix")
        if event_time < datetime(2023, 11, 28):
            client = client_idf
        else:
            client = client_usdf
        # slew = eventMaker.findEvent(Time(events_frame["time"][j], format="unix"))
        # if slew.blockInfos is None:
        #     block = None
        # else:
        # block = slew.blockInfos.blockNumber

        for key in table_dict.keys():
            query_list = []
            for col in ["private_sndStamp"] + [f"measuredForce{i}" for i in range(6)]:
                hpcols = [col]
                query_list.append(
                    getEfdData(
                        client=client,
                        topic=table_dict[key],
                        columns=hpcols,
                        begin=event_time,
                        end=event_time,
                        prePadding=10,
                        postPadding=10,
                    )
                )
            query = pd.concat(query_list, axis=1)
            if len(query) == 0:
                continue
            query["event_num"] = j
            query["seqNum"] = events_frame["seq_num"][j]
            query["event_time"] = events_frame["time"][j]
            query = add_timestamp(query)

            # query["block"] = block
            query["oscillation_bool"] = 0
            query["delta_time"] = query["snd_timestamp_utc"] - events_frame["time"][j]

            query_dict[key].append(query)
    for key in table_dict.keys():
        query_dict[key] = pd.concat(query_dict[key])
    sorted_keys = sorted(query_dict.keys())
    initial_key = sorted_keys[0]
    merged_df = query_dict[initial_key].sort_index()
    # Define your tolerance for matching
    tolerance = 0.03  # pd.Timedelta('0.03 seconds')

    # Iterate over the remaining DataFrames and merge them
    for key in sorted_keys[1:]:
        merged_df = pd.merge_asof(
            merged_df,
            query_dict[key].sort_index(),
            left_on="snd_timestamp_utc",
            right_on="snd_timestamp_utc",
            tolerance=tolerance,
            direction="nearest",
            suffixes=("", "_" + key),
        )
    merged_df.reset_index(drop=True, inplace=True)

    return merged_df

In [ ]:
def get_stats(data):
    keys = [f"measuredForce{n}" for n in range(5)]
    max_val = np.max(data[key])
    min_val = np.min(data[key])
    ptp = np.ptp(data[key])
    before_std = np.std(data.loc[(data["delta_time"].values < 0), key])
    rolling_std = data["measuredForce2"].rolling(100).std()
    max_time = data.iloc[np.argmax(data[key]), :]["delta_time"]
    sel_time = (data["delta_time"] > max_time) & (rolling_std <= before_std + 10)

    if sel_time.sum() > 0:
        duration = np.min(data.loc[sel_time, "delta_time"].values)
        return max_val, min_val, ptp, duration
    else:
        return max_val, min_val, ptp


def get_stats(data):
    # Create an empty DataFrame to store the results
    stats_df = pd.DataFrame(
        columns=["name", "Max Value", "Min Value", "Peak-to-Peak", "Duration"]
    )

    # Define the keys
    keys = [f"measuredForce{n}" for n in range(6)]
    rows = []
    for key in keys:
        sel_0 = data["delta_time"] > 0
        max_val = max(data.loc[sel_0, key], key=abs)
        # min_val = np.min(data.loc[sel_0, key])
        ptp = np.ptp(data.loc[sel_0, key])
        before_std = np.std(data.loc[(data["delta_time"].values < 0), key])
        before_mean = np.mean(data.loc[(data["delta_time"].values < 0), key])
        rolling_std = data[key].rolling(100).std()
        max_time = data.iloc[np.argmax(data[key]), :]["delta_time"]
        sel_time = (
            (data["delta_time"] > max_time) & (rolling_std <= before_std * 2) & sel_0
        )

        # Calculate duration if condition is met
        if sel_time.sum() > 0:
            duration = np.min(data.loc[sel_time, "delta_time"].values)
        else:
            duration = np.nan  # Use NaN for cases where the condition is not met

        # Create a row dictionary and append to the list
        row = {
            "key": key,
            "max": max_val,
            "mean": before_mean,
            "ptp": ptp,
            "duration": duration,
        }
        rows.append(row)
    stats_df = pd.DataFrame(rows)

    return stats_df

In [ ]:
event

In [ ]:
client_usdf = EfdClient("usdf_efd")
client_idf = EfdClient("idf_efd")
events_frame = pd.read_csv(
    "./sitcomtn81_data/231121_dynamic_testing_cleaned_oscillations.csv"
)
dayObs = 20231205
events_frame = events_frame  # [events_frame["day_obs"] == dayObs].reset_index()
merged_df = get_data(events_frame, client_usdf, client_idf)

In [ ]:

keep_bool = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
keep_bool += [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
keep_bool += [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
keep_bool += [0, 0, 1, 1, 0, 0, 0, 0, 0, 1]

keep_bool += [1, 1, 0, 0, 1, 1, 0, 0, 0, 0]

keep_bool += [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
keep_bool += [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
keep_bool += [0, 1, 1, 0, 1]
keep_bool = keep_bool + list(np.ones(len(events_frame) - len(keep_bool)))

In [ ]:
events_frame["event_num"] = np.arange(len(events_frame))

In [ ]:
real_events_frame = events_frame[np.array(keep_bool) == 1]

In [ ]:
offsets = [-0.1, -1.5, -4, 1, 6, -2, -2, -1.5, 4.5, -2, -2]
offsets += [-2, -2]
real_events_frame.loc[:, "offset"] = offsets + list(
    np.zeros(len(real_events_frame) - len(offsets))
)

In [ ]:
real_events_frame.columns

In [ ]:
# l = 4  # should be 1 or 4
plt.figure(dpi=175, figsize=(12, 8))
for i in real_events_frame["event_num"]:
    # plt.title(f"{day}, {k}, {l}, {k/l/30}")
    subframe = merged_df.copy()
    subframe = subframe[subframe["event_num"] == i]
    seq_val = np.unique(subframe["seqNum"])
    dayObs = str(
        real_events_frame.loc[real_events_frame["event_num"] == i, "day_obs"].values[0]
    )
    if len(subframe) > 0:
        offset = float(
            real_events_frame.loc[real_events_frame["event_num"] == i, "offset"].values
        )
        plt.plot(
            subframe["delta_time"] - offset,
            subframe["measuredForce2"] - 800 * j,
            label=f"{dayObs}, {seq_val[0]}",
        )
        j += 1
if l == 1:
    plt.legend()
if l == 4:
    plt.legend(fontsize=9.2, loc=7, title="seqNum, eventNum")
plt.axvline(0, c="k", ls="dashed")
plt.axvline(1, c="r", ls="dashed")
plt.axvline(-1, c="r", ls="dashed")
# plt.savefig(f"./sitcomtn81_data/plots/diagnostic_{day}_{k}_{l}_{int(k/l/30)}.png")
# plt.close()

In [ ]:
df_list = []
for i in np.unique(merged_df["event_num"]):
    sel = merged_df["event_num"] == i
    stats_df = get_stats(merged_df[sel])
    stats_df["day_obs"] = events_frame["day_obs"].values[i]
    stats_df["seq_num"] = events_frame["seq_num"].values[i]
    # stats_df["elevation_velocity"] = events_frame['elevation_velocity'].values[i]
    stats_df["time"] = events_frame["time"].values[i] + 3
    df_list.append(stats_df)
stats_df = pd.concat(df_list)